# Segmenting and Clustering Neighborhoods in Toronto

### 1. Requirements

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests  # library to handle requests

# Installing the BeautifulSoup package
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forg

### 2. Scraping data

In [7]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,"html.parser")
table = soup.find_all('table')[0] 

# Scrape table and pass data to array
board_members = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    if cols != []:
        board_members.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()))
board_array = np.asarray(board_members)

# Convert array to dataframe
df = pd.DataFrame(board_array)
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


### 3. Cleaning Data

In [22]:
# Ignoring cells with a borough that is Not assigned
df.drop(df[df.Borough == 'Not assigned'].index, inplace = True)
df.reset_index(drop=True, inplace=True)

# Neighborhood with Not assigned value
for i in range(df.shape[0]):
    if df.loc[i, 'Neighborhood'] == 'Not assigned':
        df.loc[i, 'Neighborhood'] = df.loc[i, 'Borough']

# Combinning rows
j = 0
while j < df.shape[0]-1:
    if df.loc[j, 'PostalCode'] == df.loc[j+1, 'PostalCode']:
        df.loc[j, 'Neighborhood'] = df.loc[j, 'Neighborhood'] + ", " + df.loc[j+1, 'Neighborhood']
        df.drop([j+1], axis = 0, inplace = True)
        df.reset_index(drop=True, inplace=True)
        j = j-1
    j = j+1

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [23]:
# Number of rows
df.shape

(103, 3)